In [40]:
import sys
sys.path.append('../')
import wsgi
import pickle as pkl
from flask import jsonify
import pandas as pd

In [68]:
    df_final = pkl.load(open('../app/static/data/df_final.pkl','rb'))
    il = 'Antalya' #request.args.get('il','xxx')
    urun = 'Maviyemiş' #request.args.get('urun','xxx')
    #'rating_actual':'Gerçek Verim'
    df = df_final[(df_final.Il_Adi==il) & ((df_final.Urun_Adi==urun) | (urun==''))] \
      .sort_values(['prediction','rating'], ascending=[True,False]) \
      .loc[:,['Ilce_Adi','Urun_Adi','Uretim_Miktar','Uretim_Olcu','Verim','Verim_Olcu','prediction','rating',]] \
      .rename(columns={'prediction':'Tahmin','rating':'Verim Oranı',}) \
      .assign(Tahmin = df_final.prediction.replace({1:'tahmin', 0:'üretilen'}))
    tahmin = pd.IndexSlice[df.loc[~(df['Tahmin']=='tahmin')].index, 'Verim Oranı']
    gercek = pd.IndexSlice[df.loc[~(df['Tahmin']=='üretilen')].index, 'Verim Oranı']
    ret = ({
      'tablo1':df,#.to_html(index=False, classes='mystyle'),
      'tablo2':df \
            .style.format('{:,.2f}', subset=['Uretim_Miktar','Verim','Verim Oranı']) \
            .applymap(lambda x: "color:white", subset=tahmin) \
            .bar( color="rgba(21,142,165,190)", subset=tahmin) \
            .bar( color="rgba(21,192,225,200)", subset=gercek) \
            #.render().replace('nan','')
    })
    display(ret['tablo2'])

,Ilce_Adi,Urun_Adi,Uretim_Miktar,Uretim_Olcu,Verim,Verim_Olcu,Tahmin,Verim Oranı
16869,Korkuteli,Maviyemiş,673.00,Ton,"1,496.00",Kg/Dekar,üretilen,1.00
28474,Finike,Maviyemiş,nan,nan,nan,nan,tahmin,0.75
31441,Kepez,Maviyemiş,nan,nan,nan,nan,tahmin,0.69
31952,Demre,Maviyemiş,nan,nan,nan,nan,tahmin,0.68
39723,Kumluca,Maviyemiş,nan,nan,nan,nan,tahmin,0.62
39734,Serik,Maviyemiş,nan,nan,nan,nan,tahmin,0.62
41101,Kaş,Maviyemiş,nan,nan,nan,nan,tahmin,0.61
42052,Aksu,Maviyemiş,nan,nan,nan,nan,tahmin,0.60
49733,Konyaaltı,Maviyemiş,nan,nan,nan,nan,tahmin,0.56
52555,Muratpaşa,Maviyemiş,nan,nan,nan,nan,tahmin,0.55
